In [5]:
from dcicutils import ff_utils
from functions.wfr import *
from functions.wfr_settings import *
from functions.notebook_functions import *

# tibanna = Tibanna(env=env)
my_env = 'data'
my_auth = get_key('koray_data')

# all usable env names
all_envs = ['data', 'staging', 'fourfront-webdev', 'fourfront-mastertest', 'fourfront-hotseat']
# #get admin key from s3
# my_env = 'fourfront-webdev'
# my_auth = ff_utils.get_authentication_with_server({}, ff_env = my_env)


# CHIP RUNS
set_url = '/search/?experiments_in_set.experiment_type=ChIP-seq&type=ExperimentSetReplicate&limit=all&status=pre-release&status=released&status=released%20to%20project'

set_url = '/search/?award.project=4DN&experiments_in_set.experiment_type=ChIP-seq&experimentset_type=replicate&lab.display_title=Feng+Yue+Lab%2C+PSU&type=ExperimentSetReplicate&status=pre-release&status=released&status=released%20to%20project'
# CUT&RUN RUNS
# set_url = '/search/?experiments_in_set.experiment_type=CUT%26RUN&type=ExperimentSetReplicate&limit=all&status=pre-release&status=released&status=released%20to%20project'

all_sets = ff_utils.search_metadata(set_url , key=my_auth)
counter = 0
completed = 0
completed_acc = []

run_sets = [i for i in all_sets if "ENCODE_ChIP_Pipeline_1.1.1"  not in i.get('completed_processes', [])]
print(len(all_sets), 'total number of sets')
print(len(all_sets)-len(run_sets), 'sets completed')
print(len(run_sets), 'ready for processing')

4 total number of sets
0 sets completed
4 ready for processing


In [6]:
run_wfr = False
add_pc = True
add_tag = True

for a_set in run_sets:      
    print()
    print(a_set['accession'], end = " ")
    counter += 1
    # some feature to extract from each set
    control = ""  # True or False (True if set in scope is control)
    control_set = ""  # None (if no control exp is set), or the control experiment for the one in scope
    target_type = "" # Histone or TF (or None for control)
    paired = "" # single or paired
    organism = ""
    
    # pass attributions to new objects
    attributions = None
    
    replicate_exps = a_set['replicate_exps']
    replicate_exps = sorted(replicate_exps, key = lambda x: [x['bio_rep_no'], x['tec_rep_no']])

    # get organism, target and control from the first replicate
    f_exp = replicate_exps[0]['replicate_exp']['uuid']
    f_exp_resp = ff_utils.get_metadata(f_exp, key = my_auth)
    control, control_set, target_type, organism = get_chip_info(f_exp_resp, my_auth)
           
    print(organism, paired, control, control_set)  

    if organism not in ['mouse', 'human']:
        print('orgamism not ready', organism)
        continue

    attributions = get_attribution(ff_utils.get_metadata(f_exp_resp['files'][0]['uuid'], key = my_auth))
    
    if not control:
        if not target_type:
            print('set is not control, but missing target type, skipping')
            continue

    ta = []
    taxcor = []
    # check for step1, and start if missing
    step1_status = 'Done'

    for an_exp in replicate_exps:
        exp_id = an_exp['replicate_exp']['accession']
        exp_resp = ff_utils.get_metadata(exp_id, my_auth)
        
        run_name = exp_resp['accession']
        print(run_name, end = ' status: ')
        exp_files, exp_obj, paired = get_chip_files(exp_resp, my_auth)
        if len(exp_files) > 2:
            print('WARNING More then 2 seq reps, Soo needs to have a look before we can proceed')
            print('SKIPPING')
            continue
            
        if control:
            step1_result = get_wfr_out_file(exp_files[0][0], 'encode-chipseq-aln-ctl 1.1.1', my_auth)
            print(step1_result['status'])
            if step1_result['status'] == 'complete':
                if add_pc:
                    add_preliminary_processed_files(exp_id, [step1_result['chip.first_ta_ctl']], my_auth, run_type = 'chip')
            elif step1_result['status'] == 'running':
                step1_status = 'Incomplete'
            else:
                step1_status = 'Incomplete'                
                if run_wfr:
                    print('starting run')
                    run_missing_chip1(control, step_settings('encode-chipseq-aln-ctl', organism, attributions), 
                                      organism, 'tf', paired, [exp_files], [exp_obj], my_env, my_auth, run_name)
        else:
            step1_result = get_wfr_out_file(exp_files[0][0], 'encode-chipseq-aln-chip 1.1.1', my_auth) 
            print(step1_result['status'])
            if step1_result['status'] == 'complete':
                ta.append(step1_result['chip.first_ta'])
                taxcor.append(step1_result['chip.first_ta_xcor'])
                if add_pc:
                    add_preliminary_processed_files(exp_id, [step1_result['chip.first_ta']], my_auth, run_type = 'chip')
            elif step1_result['status'] == 'running':
                step1_status = 'Incomplete'
            else:
                step1_status = 'Incomplete'
                if run_wfr:
                    print('starting run')
                    run_missing_chip1(control, step_settings('encode-chipseq-aln-chip', organism, attributions), 
                                      organism, target_type, paired, [exp_files], [exp_obj], my_env, my_auth, run_name)

                    
    if step1_status != 'Done':
        continue
        
    # add pc files and tag for control
    if control:
        #add competed flag to set
        if add_tag:
            ff_utils.patch_metadata({"completed_processes":["ENCODE_ChIP_Pipeline_1.1.1"]}, obj_id=a_set['accession'] , key=my_auth)
        # add processed files to set
        continue
    
    print('set ready for part2')
    # check if control is ready
    ta_cnt=[]
    control_ready = True
    cont_set_info = ff_utils.get_metadata(control_set, key=my_auth)
    replicate_exps_cnt = cont_set_info['replicate_exps']
    replicate_exps_cnt = sorted(replicate_exps_cnt, key = lambda x: [x['bio_rep_no'], x['tec_rep_no']])
    
    for an_exp_cnt in replicate_exps_cnt:
        exp_cnt_id = an_exp_cnt['replicate_exp']['accession']
        exp_cnt_resp = ff_utils.get_metadata(exp_cnt_id, my_auth)
        files_cnt, obj_key_cnt, paired_cnt = get_chip_files(exp_cnt_resp, my_auth)
        step1_result = get_wfr_out_file(files_cnt[0][0], 'encode-chipseq-aln-ctl 1.1.1', my_auth) 
        if step1_result['status'] == 'complete':
            ta_cnt.append(step1_result['chip.first_ta_ctl'])
        else:
            control_ready = False
            break

    if not control_ready:
        print('Control not ready for second step')
        continue

    if len(ta) > 2 or len(ta_cnt) > 2:
        ta_2 = []
        taxcor_2 = []
#         print('skipping more then 2 exp set')
#         continue
        print('Experiment has more then 2 replicates, selecting best 2')
        ta_2 = select_best_2(ta, my_auth)
        # xcor does not have qc, use ta indexes to find the correct files
        for ta_f in ta_2:
            taxcor_2.append(taxcor[ta.index(ta_f)])
        ta = ta_2
        taxcor = taxcor_2
        ta_cnt = select_best_2(ta_cnt, my_auth)
             
    if len(ta_cnt) != len(ta):
        print('Control and experiment have diferent number of bio reps, skipping')
        continue
    
    step2_result = get_wfr_out_file(ta[0], 'encode-chipseq-postaln 1.1.1', my_auth)
    if step2_result['status'] == 'complete':
        print('step2 is complete')
        print('https://data.4dnucleome.org/' + step2_result['chip.sig_fc'])
        if add_pc:
            add_preliminary_processed_files(a_set['accession'], 
                                            [
                                                step2_result['chip.optimal_peak'],
                                                step2_result['chip.conservative_peak'],
                                                step2_result['chip.sig_fc']
                                            ], 
                                            my_auth, run_type = 'chip')
        if add_tag:
            ff_utils.patch_metadata({"completed_processes":["ENCODE_ChIP_Pipeline_1.1.1"]}, obj_id=a_set['accession'] , key=my_auth)   
    elif step2_result['status'] == 'running':
        print('step2 is still running')
    else:
        step2_status = 'Incomplete' 
        print('missing step2')
        if run_wfr:
            print('starting run')
            run_ids = {'run_name': a_set['accession'],
                       "desc" : a_set.get('description', ''),
                      }
            run_missing_chip2(control_set, step_settings('encode-chipseq-postaln', organism, attributions), 
                              organism, target_type, paired, ta, taxcor, ta_cnt, my_env, my_auth, run_ids)



4DNESYMCM19N human  True 
4DNEXW3RWUAH status: complete
Error - Cannot add files to opc
The same title already in other processed files
4DNEXMOG6F4W status: complete
Error - Cannot add files to opc
The same title already in other processed files

4DNESSNWXHXK human  False 4DNESYMCM19N
4DNEXYQ6TC5B status: complete
Error - Cannot add files to opc
The same title already in other processed files
4DNEXV57K68O status: complete
Error - Cannot add files to opc
The same title already in other processed files
set ready for part2
step2 is complete
https://data.4dnucleome.org//files-processed/4DNFINRI6WOL/
Error - Cannot add files to opc
The same title already in other processed files

4DNESWX32ZCG human  False 4DNESYMCM19N
4DNEXYOW7TCB status: complete
Error - Cannot add files to opc
The same title already in other processed files
4DNEXBMMVBQW status: complete
Error - Cannot add files to opc
The same title already in other processed files
set ready for part2
step2 is complete
https://data.4dnuc

In [ ]:
# Move files from opc to pc
from dcicutils import ff_utils
from functions.notebook_functions import *
from functions.wfr import *

recipe_no = 1
exp_type, step3 = recipe[recipe_no]
action = False
move_title = 'HiC Processing Pipeline - Preliminary Files'

set_url = '/search/?'+ \
          '&'.join(['experiments_in_set.experiment_type='+i for i in exp_type])+ \
          '&type=ExperimentSetReplicate&limit=all' + \
          '&status=released&status=released%20to%20project'

print(set_url)
set_url = '/search/?award.project=4DN&experimentset_type=replicate&lab.display_title=Chuck+Murry%2C+UW&status=pre-release&type=ExperimentSetReplicate'

# exp
# set_url = '/search/?'+ \
#           '&'.join(['experiment_type='+i for i in exp_type])+ \
#           '&type=Experiment&limit=all' + \
#           '&status=released&status=released%20to%20project'

all_sets = ff_utils.search_metadata(set_url , key=my_auth)

ready_sets_1 = [i for i in all_sets if "HiC_Pipeline_1.1.1" in i.get('completed_processes', [])]
print(len(ready_sets_1))
ready_sets_2 = []
for a_set in ready_sets_1:
    if a_set.get('other_processed_files'):
        print('a')
        print(a_set['accession'])
        if move_title in [i['title'] for i in a_set['other_processed_files']]:
            print('b')
            if a_set.get('processed_files'):
                print('c')
                print('WARN' ,a_set['accession'], 'has items in processed files, skipping ')
                continue
            else:
                ready_sets_2.append(a_set)
print(len(ready_sets_2), 'items are ready')

In [ ]:
# move other processed files to processed files field
action = True
def move_opc_to_pc(resp, move_title, con_key):
    opc = resp.get('other_processed_files')
    pc = resp.get('processed_files')
    # if processed_files field already has values, exit
    if pc:
        if opc:
            print('There are files in processed_files field, expected empty', resp['accession'])
            return False
        else:
            print('it is possible that move already happened, no opc but pc', resp['accession'])
    # see if there are other_processed_files to move
    if opc:
        titles = [i['title'] for i in opc]
        if move_title in titles:
            print(resp['accession'], 'files will move')
            move_item = [i for i in opc if i['title'] == move_title]
            assert len(move_item) == 1
            assert move_item[0]['type'] == 'preliminary'
            new_pc = move_item[0]['files']
            new_opc = [i for i in opc if i['title'] != move_title]
            # Time to patch
            patch_data = {}
            add_on = ""
            #if there is something left in opc, patch it, if not delete field
            if new_opc:
                patch_data['other_processed_files'] = opc
            else:
                add_on = 'delete_fields=other_processed_files'
            # patch with processed files
            patch_data['processed_files'] = new_pc
            if action:
                ff_utils.patch_metadata(patch_data, resp['uuid'], key = con_key, add_on = add_on)
                # update status of pc to status of set or exp
                release_files(resp['uuid'], new_pc, con_key)
            return True
        else:
            return False

        
set_w_apf = 0
exp_w_apf = 0
counter = 0
move_title = 'HiC Processing Pipeline - Preliminary Files'

print(len(ready_sets_2), 'experiment sets in scope')
for a_set in ready_sets_2:
    set_resp = ff_utils.get_metadata(a_set['uuid'],key=my_auth, add_on='frame=raw')
    counter += 1
    print(counter, set_resp['accession'])
    exps = set_resp['experiments_in_set']
    res =  move_opc_to_pc(set_resp, move_title, my_auth)
    if res:
        set_w_apf += 1
        print(set_resp['accession'], 'moved to pc')
  
    for exp in exps:
        exp_resp = ff_utils.get_metadata(exp, key=my_auth, add_on='frame=raw')
        res_e =  move_opc_to_pc(exp_resp,move_title,my_auth)
        if res_e:
            exp_w_apf += 1
            print(exp_resp['accession'], 'moved to pc')
    print()

print(set_w_apf)
print(exp_w_apf)


4DNESYMCM19N human  True 
4DNEXW3RWUAH status: complete
Error - Cannot add files to opc
The same title already in other processed files
4DNEXMOG6F4W status: complete
Error - Cannot add files to opc
The same title already in other processed files

4DNESSNWXHXK human  False 4DNESYMCM19N
4DNEXYQ6TC5B status: complete
Error - Cannot add files to opc
The same title already in other processed files
4DNEXV57K68O status: complete
Error - Cannot add files to opc
The same title already in other processed files
set ready for part2
step2 is complete
https://data.4dnucleome.org//files-processed/4DNFINRI6WOL/

4DNESWX32ZCG human  False 4DNESYMCM19N
4DNEXYOW7TCB status: complete
Error - Cannot add files to opc
The same title already in other processed files
4DNEXBMMVBQW status: complete
Error - Cannot add files to opc
The same title already in other processed files
set ready for part2
step2 is complete
https://data.4dnucleome.org//files-processed/4DNFIVYWZE1W/
Error - Cannot add files to opc
The sam